In [ ]:
def eqn1(S):
    z = S[0] ^ S[32] ^ S[31] ^ S[7] ^ S[63] ^ S[56] ^ S[12]&S[15]   
    return z

def eqn2(A):
    z = 0
    for i in range(len(A)):
        z += (A[i]^A[i-1])
    return z
    
def update(S):                    #It will upate the value of array S through some equations
    a = S[62] ^ S[63] ^ S[23]&S[44] ^ S[15]
    b = S[16] ^ S[6] ^ S[14]&S[15] ^ S[23] 
    S = [a] + S[:31] + [b] + S[32:63]        #Injecting a and b at 0th and 32th positions in S while right shifting rest of the values
    return S

#Main function for illustration of our discussed problem
def toy(array_size, prt_len, error, Error_range, N, error_flag, array_org):
    
    N_prt = int(array_size/prt_len)                    #Number of parts after partition
    bitlen = math.floor(math.log(prt_len,2)) + 1       
    #It will decide the modulo (used for ZeroExt) in which equation has to be solved, as max value for equation 2 is prt_len if S = [0,1,0,1,0,1....]

    S = array_org[:]                           
    Z = [0]*N           #Storing the value for equation 1 on the array S and consecutively on the updated version of S    
    prt_val = [[0 for i in range(prt_len)] for i in range(N)]   #It will store the equation 2 values of each partition of S and its updated version

    for i in range(N):
        Z[i] = eqn1(S)      #Storing the value of equation 1 in Z
        
        for l in range(N_prt):
            prt_val[i][l] = eqn2(S[l*prt_len : l*prt_len+prt_len])     #Storing the value of equation 2 in prt_val for each parts of S
        S = update(S)             #update the array S


    #Putting Error randomly to the original prt_equ sequence
    if len(Error_range) != 0:
        
        if error_flag == 1:
            for i in range(N):
                for j in range(N_prt):
                    temp = random.choice(Error_range) 
                    while prt_val[i][j] + temp not in [k for k in range(prt_len)]:  #It ensures that after adding error it does not fall outside the range [0,..., prt_len-1]
                        temp = random.choice(Error_range)

                    prt_val[i][j] += temp

        if error_flag == 0:
            temp = int(0.70*N)  #Injecting error randomly for atmost 30% of the positions, thus following normal distribution

            for i in range(temp,N,1):
                for j in range(N_prt):
                    temp = random.choice(Error_range) 
                    while prt_val[i][j] + temp not in [k for k in range(prt_len)]:  #It ensures that after adding error it does not fall outside the range [0,..., prt_len-1]
                        temp = random.choice(Error_range)

                    prt_val[i][j] += temp

    
    #Formation of SMT constraints for recovery of array_org  
    m = Solver()
    arr_var_org = [BitVec('s%d'%i,1) for i in range(array_size)]   #Define 1-bit variable for original array,i.e., array_org
    S = arr_var_org[:]
    
    Z_equ = [0]*N                    #It will store the equation 1 for S and its updated version    
    prt_val_equ = [[0 for i in range(N_prt)] for i in range(N)] #It will store the equation 2 for S and its updated version
    
    
    
    for i in range(N):
        Z_equ[i] = (eqn1(S) == Z[i])       #Storing the equation 1 and equating it to its original value  

        for l in range(N_prt):
            temp = S[l*prt_len: l*prt_len + prt_len]
            for k in range(prt_len):
                prt_val_equ[i][l] += ZeroExt(bitlen-1,(temp[k]^temp[k-1]))  #Storing equation 2. 
                #ZeroExt is used to extend the size, as variable is S is of 1-bit but value of equation 2 can go upto prt_len, e.g., 16
                
        S = update(S)     #updating the array S
    m.add(Z_equ)          #Adding all equation 1 to the model m

    #Modelling the constraint: available value - error <= equation 2 (prt_val_equ) <= available value + error
    for l in range(N_prt):                
        for i in range(N):                
            m.add(UGE(prt_val_equ[i][l], max(0,prt_val[i][l]-error)))     
            m.add(ULE(prt_val_equ[i][l], min(prt_len,prt_val[i][l]+error)))  


    start = time.time()
    temp = m.check()             #Solving SMT constraints 
    end = time.time()
    
    #Extracting the solution and matching it with the original value
    print("Runtime = ",end - start)
    print("##########",temp,"##########")
    if temp == sat: 
        soln = m.model()
        out = [soln[arr_var_org[i]] for i in range(array_size)]
        if array_org == out:          
            print("**********************************Solution matched with original******************************")
        else:
            print("####### Solution not matched- Multiple solution exists.\n Increase the number of equations, i.e., increase N ######\n\n")
            

In [ ]:
import math
import random
import time

from z3 import *
array_size = 64           #Size of the input array
prt_len = 16              #Partition length  (in multiple of 8)
error = 2                 #Upper bound of the error to be injected, e.g., If error = 2, then errors to be injected are from [-2,-1,0,1,2]
N = 60                    #Number of times the array has to be updated
sample = 5
Error_range = [i for i in range(-error, error+1,1)]

print("Array size ",array_size, ", Partition length ",prt_len,", Error class ", Error_range)
print("Number of times the input array has to be updated = ",N)

for i in range(sample):
    print("\n\nTrial = ",i)
    array_org = [random.randint(0,1) for j in range(array_size)]

    error_flag = 1           #Errors injected uniformly within the range [-error,-error+1,...., error-1,error], e.g., if error =2 the range is [-2,-1,0,1,2]

    print("\nCASE 1 - Error_flag =",error_flag,", i.e., Error injected uniformly/randomly from the class [-error, ..., error]")
    toy(array_size, prt_len, error, Error_range, N, error_flag, array_org)
    
    error_flag = 0           #Error follows normal distribution. As an example, we inject error in <= 30% of the positions.
    print("\n\nCASE 2 - Error_flag =",error_flag,", i.e., error injected at <= 30% of the positions in eqn2 values")
    toy(array_size, prt_len, error, Error_range, N, error_flag, array_org)
    